In [2]:
import numpy as np
import sympy as sp
from scipy import optimize as opt
from sympy.utilities.lambdify import lambdify
from sympy import Matrix
from tkinter import *
import time

#  initialize
_wid = 700
_hei = 700
wid = 500  # view's width
hei = 500  # view's height
edge = np.genfromtxt('csv/adjacency.csv', delimiter=",").astype(np.int64)
eN = len(edge)
P = np.genfromtxt('csv/mdSpace.csv', delimiter=",")
n, d = P.shape

In [8]:
dim = 2

def genE():
    L = np.genfromtxt('csv/eigVals.csv', delimiter=",")[0:d]
    base = np.zeros(d*dim).reshape(dim,d)
    for i in range(d): base[i%dim][i] = 1
    E = base*L
    return E

E = genE()

In [10]:
Xs = np.zeros(n)
Ys = np.zeros(n)
Xs_scaled = np.zeros(n)
Ys_scaled = np.zeros(n)
boundingX = 0
boundingY = 0

def scale(pnt,bool):
	if(bool): return wid*(pnt + boundingX/2)/boundingX + (_wid - wid)/2
	else: return (hei-100)*(boundingY/2 - pnt)/boundingY + (_hei - hei)/2

def unscale(pnt,bool):
	if (bool): return boundingX * ((pnt - (_wid - wid)/2) - wid / 2) / wid
	else: return boundingY * ((pnt - (_hei - hei)/2) - (hei - 100) / 2) / (100 - hei)

def update_points():
	for i in range(n):
		global Xs, Ys, boundingX, boundingY
		p0 = P[i, 0:d]
		Xs[i] = np.dot(p0, E[0]) ; Ys[i] = np.dot(p0, E[1])
	boundingX = max([np.amax(Xs), np.amin(Xs)]) * 2
	boundingY = max([np.amax(Ys), np.amin(Ys)]) * 2
	for i in range(n):
		Xs_scaled[i] = scale(Xs[i], True);Ys_scaled[i] = scale(Ys[i], False)

update_points()

In [11]:
# sympy
a1,b1,c1,a2,b2,c2,t,s = sp.symbols('a1 b1 c1 a2 b2 c2 t s')   # variables
x2_s,y2_s = sp.symbols('x2_s y2_s')  # values
P_i = sp.MatrixSymbol('P_i', d, 1)
E1 = sp.MatrixSymbol('E1', d, 1)
E2 = sp.MatrixSymbol('E2', d, 1)
var = (x2_s,y2_s,P_i,E1,E2,a1,b1,c1,a2,b2,c2,t,s)

_E1 = sp.Matrix(a1*E1 + b1*E2 + c1*P_i)
_E2 = sp.Matrix(a2*E1 + b2*E2 + c2*P_i)
R = sp.Matrix(s*E1 + t*E2)

f = Matrix([
		_E1.dot(_E1) - 1,
		_E2.dot(_E2) - 1,
		_E1.dot(_E2),
		R.dot(R) - 1,
		_E1.dot(R) - sp.Matrix(E1).dot(R),
		_E2.dot(R) - sp.Matrix(E2).dot(R),
		sp.Matrix(P_i).dot(_E1) - x2_s,
		sp.Matrix(P_i).dot(_E2) - y2_s
		])

func = sp.Matrix.norm(f)
lam_f = lambdify(var, func, 'numpy')

def lam(x2, y2, p, e_1, e_2):
    return lambda a1,b1,c1,a2,b2,c2,t,s: \
        lam_f(x2, y2, p, e_1, e_2, a1, b1, c1, a2, b2, c2, t, s)

arr_init = np.array([1, 0, 0, 0, 1, 0, 1, 1])
print("lambda: ready")

lambda: ready


In [17]:
newPoint = sp.MatrixSymbol('newPoint', dim, 1) # values
Vars = sp.MatrixSymbol('Vars', dim+1, d)
Vars_R = sp.MatrixSymbol('Vars_R', 1, d)
Pi = sp.MatrixSymbol('P_i', d, 1)
Es = sp.MatrixSymbol('E1', d, dim)
Es_M = sp.Matrix(Es)
Pi_M = sp.Matrix(Pi)
var = (newPoint,Pi,Es,Vars,Vars_R)

_E_con = Matrix(Vars) * Es_M.col_insert(d,Pi_M)
R = Matrix(Vars_R) * Es_M

sp.simplify(R)

Matrix([[Vars_R[0, 0]*E1[0, 0] + Vars_R[0, 1]*E1[1, 0] + Vars_R[0, 2]*E1[2, 0] + Vars_R[0, 3]*E1[3, 0] + Vars_R[0, 4]*E1[4, 0] + Vars_R[0, 5]*E1[5, 0] + Vars_R[0, 6]*E1[6, 0] + Vars_R[0, 7]*E1[7, 0] + Vars_R[0, 8]*E1[8, 0] + Vars_R[0, 9]*E1[9, 0] + Vars_R[0, 10]*E1[10, 0] + Vars_R[0, 11]*E1[11, 0], Vars_R[0, 0]*E1[0, 1] + Vars_R[0, 1]*E1[1, 1] + Vars_R[0, 2]*E1[2, 1] + Vars_R[0, 3]*E1[3, 1] + Vars_R[0, 4]*E1[4, 1] + Vars_R[0, 5]*E1[5, 1] + Vars_R[0, 6]*E1[6, 1] + Vars_R[0, 7]*E1[7, 1] + Vars_R[0, 8]*E1[8, 1] + Vars_R[0, 9]*E1[9, 1] + Vars_R[0, 10]*E1[10, 1] + Vars_R[0, 11]*E1[11, 1]]])

In [ ]:
temp1 = e1
temp2 = e2
x2 = unscale(0,True)
y2 = unscale(0,False)
thisID = 14
f2 = lam(x2, y2, P[thisID].reshape(d, 1), e1, e2)
def g(args): return f2(*args)
res = opt.minimize(g, arr_init, method='L-BFGS-B',options={'ftol':1e-3})
if(res.success):
    e1 = res.x[0] * temp1 + res.x[1] * temp2 + res.x[2] * P[thisID].reshape(d, 1)
    e2 = res.x[3] * temp1 + res.x[4] * temp2 + res.x[5] * P[thisID].reshape(d, 1)
    update_points()